<a href="https://colab.research.google.com/github/dachib04/CyberSecurity_LR_Model/blob/main/Copy_of_CyberSecurity_LR_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras


In [ ]:
!pip install numpy
!pip install scikit-learn
!pip install scipy


In [ ]:
!pip install panda

In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

In [ ]:
import pandas as pd


df = pd.read_csv('/content/drive/MyDrive/folder/KDDTest.csv', header=None)
print('Read {} rows.'.format(len(df)))

df.dropna(inplace = True, axis = 1)

df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_file',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_Rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome',
    'last_flag'
]

df[0:5]

In [ ]:
ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for i in s.index:
        result.append("{}:{}%".format(i,round(100*(s[i]/t),2)))
    return '[{}]'.format(",".join(result))

def analyze(filename):
    print()
    print('Analyzing: {}'.format(filename))
    df = pd.read_csv(filename, encoding=ENCODING)
    cols = df.columns.values
    total = float(len(df))

    print('{} rows'.format(int(total)))
    for col in cols:
      uniques = df[col].unique()
      unique_count = len(uniques)
      if unique_count > 100:
          print('** {}:{} ({}%)'.format(col,unique_count,int(((unique_count)/total)*100)))
      else:
          print('** {}:{}'.format(col,expand_categories(df[col])))
          expand_categories(df[col])

In [ ]:
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

analyze('/content/drive/MyDrive/folder/KDDTest.csv')

In [ ]:
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

def encode_test_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for i in dummies.columns:
      dummy_name = "{}_{}".format(name, i)
      # dummy_name = f'{name}-{i}'
      df[dummy_name] = dummies[i]
    df.drop(name, axis=1, inplace=True)


In [ ]:
# Normalize numeric features
encode_numeric_zscore(df, 'duration')
encode_numeric_zscore(df, 'src_bytes')
encode_numeric_zscore(df, 'dst_bytes')
encode_numeric_zscore(df, 'wrong_fragment')
encode_numeric_zscore(df, 'urgent')
encode_numeric_zscore(df, 'hot')
encode_numeric_zscore(df, 'num_failed_logins')
encode_numeric_zscore(df, 'num_compromised')
encode_numeric_zscore(df, 'root_shell')
encode_numeric_zscore(df, 'su_attempted')
encode_numeric_zscore(df, 'num_root')
encode_numeric_zscore(df, 'num_file_creations')
encode_numeric_zscore(df, 'num_shells')
encode_numeric_zscore(df, 'num_access_file')
encode_numeric_zscore(df, 'count')
encode_numeric_zscore(df, 'srv_count')
encode_numeric_zscore(df, 'serror_rate')
encode_numeric_zscore(df, 'srv_serror_rate')
encode_numeric_zscore(df, 'rerror_Rate')
encode_numeric_zscore(df, 'srv_rerror_rate')
encode_numeric_zscore(df, 'same_srv_rate')
encode_numeric_zscore(df, 'diff_srv_rate')
encode_numeric_zscore(df, 'srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_count')
encode_numeric_zscore(df, 'dst_host_srv_count')
encode_numeric_zscore(df, 'dst_host_same_srv_rate')
encode_numeric_zscore(df, 'dst_host_diff_srv_rate')
encode_numeric_zscore(df, 'dst_host_same_src_port_rate')
encode_numeric_zscore(df, 'dst_host_srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_serror_rate')
encode_numeric_zscore(df, 'dst_host_srv_serror_rate')
encode_numeric_zscore(df, 'dst_host_rerror_rate')
encode_numeric_zscore(df, 'dst_host_srv_rerror_rate')

# Encode categorical features
encode_test_dummy(df, 'protocol_type')
encode_test_dummy(df, 'service')
encode_test_dummy(df, 'flag')
encode_test_dummy(df, 'land')
encode_test_dummy(df, 'logged_in')
encode_test_dummy(df, 'is_guest_login')

df.dropna(inplace=True, axis=1)
df[0:5]


x_columns = df.columns.drop('outcome')
x = df[x_columns].values
dummies = pd.get_dummies(df['outcome'])
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values



In [ ]:
x_columns = df.columns.drop('outcome')
x = df[x_columns].values
dummies = pd.get_dummies(df['outcome'])
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values


In [ ]:
df.groupby('outcome')['outcome'].count()

In [ ]:



import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
df = pd.read_csv('/content/drive/MyDrive/folder/KDDTest.csv', header=None)

# Assign column names to the dataframe
col_names = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
             'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
             'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
             'is_hot_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
             'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
             'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
             'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
             'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'outcome', 'last_flag']

df.columns = col_names

# Drop columns that have constant values or unique for each record
df.drop(['num_outbound_cmds', 'is_hot_login', 'is_guest_login'], axis=1, inplace=True)

# Filter for non-normal outcomes
attacks = df[df['outcome'] != 'normal']
attack_types = attacks['outcome'].unique()

print(f"Types of cyber attacks in the dataset: {', '.join(attack_types)}")

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['protocol_type', 'service', 'flag'])

# Split the data into train and test sets
X = df.drop('outcome', axis=1)
y = df['outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the numerical variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a logistic regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions on the test set and calculate accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print the confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{confusion_mat}")



Types of cyber attacks in the dataset: neptune, saint, mscan, guess_passwd, smurf, apache2, satan, buffer_overflow, back, warezmaster, snmpgetattack, processtable, pod, httptunnel, nmap, ps, snmpguess, ipsweep, mailbomb, portsweep, multihop, named, sendmail, loadmodule, xterm, worm, teardrop, rootkit, xlock, perl, land, xsnoop, sqlattack, ftp_write, imap, udpstorm, phf
Accuracy: 0.9756061502069782
Confusion matrix:
[[211   0   0 ...   0   0   0]
 [  0  88   0 ...   0   0   0]
 [  0   0   4 ...   0   0   0]
 ...
 [  0   0   0 ...   2   0   0]
 [  0   0   0 ...   1   0   0]
 [  0   0   1 ...   0   0   1]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
